<a href="https://colab.research.google.com/github/rhiandoy/Amazon_Vine_Analysis/blob/main/Vine_Review_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,622 B]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-13 01:12:00--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-04-13 01:12:01 (6.46 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Beauty_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|

In [ ]:
# filter data and create a new dataframe where total_votes >=  20
vine_votes_df = df.filter(df.total_votes >= 20)
vine_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22393078|R2QRUE9REK8OUC|B00461F4PA|     608719013|Baby Foot Exfolia...|          Beauty|          5|           18|         23|   N|                Y|Better Than Any Pedi|First off, I'll s...| 2015-08-31|
|         US|   46053231|R2POXRW6PXHXZG|B00OCJZ0GE|     717345177|Bed Head Little T...|          Beauty|          5|    

In [ ]:
# filter data and create a new dataframe where helpful_votes / total_votes >= 50%

helpful_df = vine_votes_df.filter(vine_votes_df['helpful_votes']/vine_votes_df["total_votes"] >= 0.5)
helpful_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22393078|R2QRUE9REK8OUC|B00461F4PA|     608719013|Baby Foot Exfolia...|          Beauty|          5|           18|         23|   N|                Y|Better Than Any Pedi|First off, I'll s...| 2015-08-31|
|         US|   46053231|R2POXRW6PXHXZG|B00OCJZ0GE|     717345177|Bed Head Little T...|          Beauty|          5|    

In [ ]:
# create a new dataframe that retrieves all rows where a review vine == 'Y'
paid_vine_df = helpful_df.filter("vine == 'Y' ")
paid_vine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   47770601|R106V6GUNWRSSS|B00YAR86BY|     136411552|Philips Sonicare ...|          Beauty|          5|          857|        889|   Y|                N|Improved build, g...|According to spec...| 2015-08-27|
|         US|   52846213|R37N8KZS48W36A|B00YJJXJ06|     791197070|Maybelline New Yo...|          Beauty|          5|    

In [ ]:
# Repeat previous dataframe but for vine == "N"

unpaid_vine_df = helpful_df.filter("vine == 'N' ")
unpaid_vine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22393078|R2QRUE9REK8OUC|B00461F4PA|     608719013|Baby Foot Exfolia...|          Beauty|          5|           18|         23|   N|                Y|Better Than Any Pedi|First off, I'll s...| 2015-08-31|
|         US|   46053231|R2POXRW6PXHXZG|B00OCJZ0GE|     717345177|Bed Head Little T...|          Beauty|          5|    

In [ ]:
# determine total number of reviews for Paid

total_reviews_paid = paid_vine_df.count()
total_reviews_paid

647

In [ ]:
# Paid 5 star reviews

five_star_paid = paid_vine_df.filter("star_rating == 5").count()
five_star_paid

229

In [ ]:
# Paid 5 star Review Percentage

five_star_paid_percent = (five_star_paid / total_reviews_paid) * 100
five_star_paid_percent

35.394126738794434

In [ ]:
# Determine total number of reviews for Unpaid

total_reviews_unpaid = unpaid_vine_df.count()
total_reviews_unpaid

74113

In [ ]:
# Unpaid 5 star reviews

five_star_unpaid = unpaid_vine_df.filter("star_rating == 5").count()
five_star_unpaid

43217

In [ ]:
# Unpaid 5 star review Percentage

five_star_unpaid_percent = (five_star_unpaid / total_reviews_unpaid) * 100
five_star_unpaid_percent

58.312306882733125